In [4]:
import sys, os, fnmatch

import pandas as pd
import numpy as np
import sys
import random
import math

from trackml.dataset import load_event
from trackml.dataset import load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event

import seaborn as sns

sys.path.append('/home/silvio/homeremoto/headtop/git/track-ml-1/utils')
from tracktop import *
sys.path.append('/home/silvio/homeremoto/headtop/git/track-ml-1/')
from lib_data_manipulation import *

In [5]:
def createInput(dirParam,fileParam,outputTracks):
    #9 columns for particle
    #7 columns for hit
    #one column for cell value (energy deposited)

    #9+20*8 = 169
    
    hits, cells, particles, truth = load_event(os.path.join(dirParam,fileParam))
    
    columns_total_Each_track=169
    amount_of_hits=20

    dfInhttp://headtop:8080/user/silvio/notebooks/git2/track-ml-1/create_Input_data_set.ipynb#putforNN = pd.DataFrame()
    rowfinalmatrix=np.zeros((1,1))

    total_discarded_hits=0

    for index, row in particles.iloc[:,:].iterrows():
    
        dataP=row.values.flatten()

        #particle information 9 columns
        par   = np.reshape(dataP,(1,9))

        #obtain particle track
        truth_0 = truth[truth.particle_id == row['particle_id']]
    
        PriorPosition=np.zeros(3)    
        rowfinalallhits = par
        discarded_hit=0
        
        #loop all hits in track
        for indexT, rowT in truth_0.iloc[:,:].iterrows():
       
            #obtain geometric distance, between hit and previous hit
            if ((PriorPosition[0] !=0) & (PriorPosition[1] !=0) & (PriorPosition[2] !=0)):
                geodist=(( (rowT['tx']-PriorPosition[0]) **2) + ((rowT['ty']-PriorPosition[1]) **2) + 
              ((rowT['tz'] - PriorPosition[2]) **2) )
                geodist = math.sqrt(geodist)
            else:
                geodist = 100
            
            # if geometric distance is below 20 mm discard hit
            if (geodist > 20):
                #thruth information 9 columns
                data=rowT.values.flatten()
                hit   = np.reshape(data,(1,9))
            
                hit_0=hits[hits.hit_id == rowT['hit_id']]
                cells_0 = cells[cells.hit_id == rowT['hit_id']]

                sumCellsaux = cells_0.iloc[:,3:4].sum()
                dataC       = sumCellsaux.values.flatten()
                sumCell     = np.reshape(dataC,(1,1))
            
                rowfinal=np.concatenate((hit_0.iloc[:,0:8] , sumCell),axis=1)
                
                #print("rowfinal " , rowfinal, " sumCell ", sumCell)
                
                #create row with particle and all hits
                rowfinalallhits = np.concatenate((rowfinalallhits,rowfinal),axis=1)
            
                PriorPosition[0]=rowT['tx']
                PriorPosition[1]=rowT['ty']
                PriorPosition[2]=rowT['tz']
            else:
                discarded_hit=discarded_hit+1
    
        
        total_discarded_hits=total_discarded_hits+discarded_hit
        
        #complete empty hits in track with zeros 
        auxzeros=np.zeros(columns_total_Each_track-rowfinalallhits.shape[1])
        zerosdata=auxzeros.flatten()        
        autocompletewithzeros = np.reshape(zerosdata,(1,(columns_total_Each_track-rowfinalallhits.shape[1])))
    
        #create one line with complete row
        rowfinalallhits2=np.concatenate((rowfinalallhits,autocompletewithzeros),axis=1)
        
        if ((rowfinalmatrix.shape[0] == 1) & (rowfinalmatrix.shape[1] == 1)):
            rowfinalmatrix = rowfinalallhits2
        else:
            rowfinalmatrix = np.concatenate((rowfinalmatrix,rowfinalallhits2),axis=0)
        
        
    dfInputforNN = pd.DataFrame(rowfinalmatrix)
    dfInputforNN.to_csv(outputTracks, index = False)

    print(dfInputforNN.shape)
    print("total_discarded_hits " , total_discarded_hits)

In [6]:
import glob

files = glob.glob('/home/silvio/homeremoto/headtop/trainExperiments/*')
for f in files:
    os.remove(f)
    
files = glob.glob('/home/silvio/homeremoto/headtop/trainInput/train_1/*-particles.csv', recursive=True) 

tot=25
k=0
for f in files:
    flaux=os.path.basename(f)
    fl=flaux.split('-')
    print(fl[0])
    print(type(fl[0]))
    print(k)
    res="/home/silvio/homeremoto/headtop/trainExperiments/"+fl[0]+".csv"
    createInput("/home/silvio/homeremoto/headtop/trainInput/train_1/", fl[0], res)
    k=k+1
    if (k == tot):
        break
    
files = glob.glob('/home/silvio/homeremoto/headtop/trainExperiments/*.csv', recursive=True) 

dfInputforNN = pd.DataFrame()
for f in files:
    print(f)
    totdf = pd.read_csv(f) 
    
    frames = [dfInputforNN, totdf]

    result = pd.concat(frames, ignore_index=True)
    dfInputforNN=result

#dfInputforNN.drop('Unnamed: 0', axis=1, inplace=True)
print(dfInputforNN.shape)
print(dfInputforNN)
dfInputforNN.to_csv("/home/silvio/homeremoto/headtop/all-Train.csv", index = False)
#dfInputforNN.to_csv("/home/silvio/homeremoto/headtop/all-Inf.csv", index = False)

files = glob.glob('/home/silvio/homeremoto/headtop/trainExperiments/*')
for f in files:
    os.remove(f)
    
files = glob.glob('/home/silvio/homeremoto/headtop/trainInput/train_2/*-particles.csv', recursive=True) 

tot=25
k=0
for f in files:
    flaux=os.path.basename(f)
    fl=flaux.split('-')
    print(fl[0])
    print(type(fl[0]))
    print(k)
    res="/home/silvio/homeremoto/headtop/trainExperiments/"+fl[0]+".csv"
    createInput("/home/silvio/homeremoto/headtop/trainInput/train_2/", fl[0], res)
    k=k+1
    if (k == tot):
        break
    
files = glob.glob('/home/silvio/homeremoto/headtop/trainExperiments/*.csv', recursive=True) 

dfInputforNN = pd.DataFrame()
for f in files:
    print(f)
    totdf = pd.read_csv(f) 
    
    frames = [dfInputforNN, totdf]

    result = pd.concat(frames, ignore_index=True)
    dfInputforNN=result

#dfInputforNN.drop('Unnamed: 0', axis=1, inplace=True)
print(dfInputforNN.shape)
print(dfInputforNN)
#dfInputforNN.to_csv("/home/silvio/homeremoto/headtop/all-Train.csv", index = False)
dfInputforNN.to_csv("/home/silvio/homeremoto/headtop/all-Inf.csv", index = False)

event000001723
<class 'str'>
0
(11011, 169)
total_discarded_hits  22961
event000002460
<class 'str'>
1
(10451, 169)
total_discarded_hits  21820
event000001333
<class 'str'>
2
(10265, 169)
total_discarded_hits  21367
event000002051
<class 'str'>
3
(13190, 169)
total_discarded_hits  27257
event000001425
<class 'str'>
4
(9315, 169)
total_discarded_hits  19406
event000001000
<class 'str'>
5
(12263, 169)
total_discarded_hits  24943
event000001172
<class 'str'>
6
(10090, 169)
total_discarded_hits  20620
event000001302
<class 'str'>
7
(13420, 169)
total_discarded_hits  27927
event000002619
<class 'str'>
8
(9136, 169)
total_discarded_hits  18696
event000002220
<class 'str'>
9
(9161, 169)
total_discarded_hits  18800
event000002173
<class 'str'>
10
(10656, 169)
total_discarded_hits  21857
event000001192
<class 'str'>
11
(11165, 169)
total_discarded_hits  22953
event000002204
<class 'str'>
12
(12297, 169)
total_discarded_hits  25397
event000002224
<class 'str'>
13
(9022, 169)
total_discarded_hits

event000004170
<class 'str'>
0
(11690, 169)
total_discarded_hits  24244
event000003629
<class 'str'>
1
(11625, 169)
total_discarded_hits  23696
event000003620
<class 'str'>
2
(10489, 169)
total_discarded_hits  21238
event000004559
<class 'str'>
3
(11162, 169)
total_discarded_hits  23051
event000003255
<class 'str'>
4
(10054, 169)
total_discarded_hits  20440
event000003139
<class 'str'>
5
(11463, 169)
total_discarded_hits  23834
event000002916
<class 'str'>
6
(12492, 169)
total_discarded_hits  25928
event000003432
<class 'str'>
7
(8577, 169)
total_discarded_hits  17593
event000003838
<class 'str'>
8
(11540, 169)
total_discarded_hits  24270
event000004558
<class 'str'>
9
(12164, 169)
total_discarded_hits  25112
event000003197
<class 'str'>
10
(10152, 169)
total_discarded_hits  21314
event000003374
<class 'str'>
11
(10566, 169)
total_discarded_hits  21898
event000003016
<class 'str'>
12
(10781, 169)
total_discarded_hits  22756
event000003916
<class 'str'>
13
(12928, 169)
total_discarded_h